# 1. Brief Jupyter overview

## 1.1 Installation, environments; get oriented

__Question:__ What is the difference between `pip install`, `virtualenv`, and Conda (or Miniconda, Anaconda)?

__Answer:__ They're different tools to manage Python package and software installation.  They determine where Python lives, and where packages like `numpy` and `astropy` are installed & then later "searched for", whenever you ask Python to import a library.

Let's open a terminal to see where Python and its packages are installed, using the following bash commands.

        which python
        which ipython
        which jupyter

        ipython
        >>> import numpy
        >>> numpy.__file__
        >>> numpy.__version__
        >>> exit

In [ ]:
import astropy
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [ ]:
plot_params= {
    'figure.figsize': (10,8),
    'axes.labelsize': 20,
    'axes.grid': True,
    'grid.alpha': 0.25,
    'xtick.labelsize': 20,
    'ytick.labelsize': 20,
    'legend.fontsize': 15,
    'xtick.minor.visible': True,
    'ytick.minor.visible': True,
    'xtick.major.size': 7,
    'xtick.minor.size': 3,
    'ytick.major.size': 7,
    'ytick.minor.size': 3
}

plt.rcParams.update(plot_params)

## 1.2 Using and navigating Jupyter

* Code and text cells.
* Command vs. insert mode: mouse click, `Esc`, `Return`
* Execute cell: `Shift`+`Return`
* Keyboard shortcuts in command mode: `a`, `b`, `x`, `c`, `v`, `dd`, `z`
* Comment out code in bulk: `Cmd` (or `Ctrl`) + `/`
* "Restart kernel and run all cells"

In [ ]:
x = 10

In [ ]:
y = 1

In [ ]:
x + y  # during interactive Jupyter or IPython session, this is the same as print(x+y)

In [ ]:
y = 20

__Question:__ what happens if I delete all definitions of `x` and `y` above, then try to compute `x + y`?

In [ ]:
x + y


## 1.3 Useful tricks

Markdown allows rich content: images, equations, HTML.  Example of a LaTeX equation that you can edit (double click this cell to see source code):

$$
  \sum_{n=0}^{\infty} \frac{1}{x^n} = \frac{1}{1-1/x}
$$

The IPython kernel within Jupyter has many useful tricks.  Let's look at a few:
* `?` to get help, `??` to look at source.
* `%history` ("%" commands are called "line magics")
* `%%timeit` to benchmark your code ("%%" commands are called "cell magics")
* `%load_ext autoreload`

In [ ]:
%history -l 5

In [ ]:
range?

In [ ]:
for x in range(10):
    print(x)

In [ ]:
%%timeit
x = 0
for y in range(10000):
    x += y
#print(x)

In [ ]:
%%timeit
x = sum(range(10000))

In [ ]:
import my_module

In [ ]:
my_module.foobar?

In [ ]:
my_module.foobar??

In [ ]:
my_module.foobar(10)

In [ ]:
%load_ext autoreload

%autoreload 2

## 1.4 Other notes

* How to run Jupyter notebook from a remote computer (e.g., if you have many GB or TB of data that won't fit on your computer): https://confluence.columbia.edu/confluence/display/rcs/Ginsburg+-+Job+Examples#GinsburgJobExamples-JupyterNotebooks

## 1.5 References and Further Reading

* IPython magic commands (`%` and `%%`): https://ipython.readthedocs.io/en/stable/interactive/magics.html

* IPython autoreload: https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html

# 2. Getting the most out of NumPy 

Python, in general, is slow; compared to compiled languages like C and Fortran, Python is woefully inefficient. But, this is where the magic of NumPy come into play: by wrapping C data structures and methods in Python commands, NumPy drastically speeds up array operations and numerical calculations. Learning to use NumPy properly can do wonders for your Python code.

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt

## Python lists vs. NumPy arrays: when to use which?

In [ ]:
# appending to python lists is faster than appending to NumPy arrays!

l1 = []
a1 = np.array([])

# append 1000 elements to a list
t1 = time.time()
for i in range(1000):
    l1.append(i)
t2 = time.time()

# append 1000 elements to a NumPy array
t3 = time.time()
for i in range(1000):
    a1 = np.append(a1, np.array(i))
t4 = time.time()

print(f"List time: {t2-t1}; NumPy array time: {t4-t3}")  # more than an order of magnitude difference!

In [ ]:
# Thankfully, python lists and NumPy arrays can be interconverted (in most cases)

converted_array = a1.tolist()
converted_list = np.array(l1)
print(f"Converted array size: {len(converted_array)}; Converted list size: {np.size(converted_list)}")

In [ ]:
# list "arithmetic" can quickly build long lists (while NumPy array arithmetic just applies a function to an array)

# some tricks for adding elements to python lists

l2 = [1, 3, 4, 5]

l2_more = l2 + [6, 7, 8]  # concatenate two lists with +
l2_triple = l2 * 3     # copy a list 3 times using *
l2.insert(1, 2)      # insert an element into index 1 of the list
print(l2_more, l2_triple, l2)

string_list = ['cat', 'dog', 'mouse']    # the above operations also work with other data types, like strings!
string_list_double = string_list * 2
string_list += ['cat'] * 4   # append 'cat' to the original string_list 4 times
print(string_list_double, string_list)


# try the same stuff with a NumPy array

a2 = np.array([1,3,4,5])
a2 = a2 * 2    # multiplies each element by 2
print(a2)

string_array = np.array(['cat', 'dog', 'mouse'])
string_array = string_array * 2    # raises an error; strings generally shouldn't be used in NumPy arrays

In [ ]:
# python lists can hold multiple different data types (including other lists), while NumPy arrays are meant to be *homogeneous*

l3 = [4, "cat", "dog", ["meow"]]
print(l3[0], l3[3])
l3 = l3 * 2
print(l3)

bad_array1 = np.array(l3)    # throws a warning about creating an array with a nested list
bad_array2 = np.array([4, "cat", "dog"])
bad_array2 = bad_array2 * 2  # throws an error because array isn't homogeneous

## NumPy array operations are *much* faster than loops

In [ ]:
# example: summing elements in an array

sum_array = np.arange(10000)  # create the array [0, 1, 2, 3, ..., 9999] (could also use linspace)

t1 = time.time()
sum1 = np.sum(sum_array)   # sum all elements in sum_array using a NumPy array operation
t2 = time.time()

sum2 = 0
t3 =  time.time()
for i in range(sum_array.size):
    sum2 += sum_array[i]    # sum all elements in sum_array using a for-loop 
t4 = time.time()

print(sum1, sum2)
print(f"NumPy operation time: {t2-t1}; Loop time: {t4-t3}")    # an order of magnitude difference in time!

In [ ]:
# example: listing an integer's factors

n = 1345986
possible_factors = np.arange(1, n+1)  # create the array [1, 2, 3, ..., n]

# try using NumPy arrays
t1 = time.time()
remainders = n % possible_factors  # modulo (%) acts on the entire possible_factors array
# print(remainders)
factors_np = possible_factors[np.where(remainders == 0)]  # pick out the elements of possible_factors that coincide with 0s in the remainders array
t2 = time.time()
print(factors_np)
print("\n")

# try using a for-loop over a python list
factors_list = []
t3 = time.time()
for p in possible_factors:
    if (n % p == 0): factors_list.append(p)
t4 = time.time()
print(factors_list)

print(f"\n NumPy operation time: {t2-t1}; Loop time: {t4-t3}")  # huge time difference!

### Getting rid of for-loops can take some finesse... some strategies:

### Strategy 1: always check for built-in NumPy functions

In [ ]:
# some useful functions (by no means an exhaustive list)

test_array = np.array([0, 0, 1, 5, 8, np.nan, 14])

where_nonzeros = np.nonzero(test_array)  # check which array entries are non-zero
print(where_nonzeros)

where_nans = np.isnan(test_array)  # check which array entries are NaN (i.e., not a number, likely resulting from division by 0)
print(where_nans)

array_max = np.max(test_array)        # find the maximum element of test_array
array_nanmax = np.nanmax(test_array)  # find the maximum element of test_array, ignoring NaNs
where_max = np.nanargmax(test_array)  # find the index of the maximum element of test_array, ignoring NaNs
print(array_max, array_nanmax, where_max)

# + lots of linear algebra stuff: https://numpy.org/doc/stable/reference/routines.linalg.html
# + lots of array manipulation functions: https://numpy.org/doc/stable/reference/routines.array-manipulation.html

### Strategy 2: use slicing and masking

In [ ]:
# some masking examples

a3 = np.array([0, 1, 3, 5, 8, 0, 10])

mask = a3 > 5            # array of booleans, True if an element of a3 is greater than 5, false otherwise
masked_a3 = a3[mask]                  # apply mask to a3 to pick out only the elements that are greater than 5
print(masked_a3)

nozeros_a3 = a3[np.nonzero(a3)]    # many built-in numpy functions create specialized masks
print(nozeros_a3)


# np.where provides an alternative means of masking and manipulating arrays

masked_a3_alt = a3[np.where(a3 > 5)]       # create a mask with np.where and apply it immediately
altered_a3 = np.where(a3 > 5, 3*a3, a3/3)  # if an element in a3 is greater than 5, replace it with thrice that element; otherwise divide the element by 3
print(masked_a3_alt, altered_a3)

In [ ]:
# some more masking examples

another_array = np.array([10, 9, 15, 37, 5, 9, 3])

cross_masked = a3[np.where(another_array % 5 == 0)]   # build and apply a mask from another array
print(cross_masked)

scrambled_array = another_array[[6, 4, 5, 0, 1, 3, 2]]   # can also use indices as a mask
print(scrambled_array)

sort_indices = np.argsort(another_array)    # get the indices that would put another_array in sorted order
sorted_array = another_array[sort_indices]  # sort another_array using sort_indices
print(sort_indices, sorted_array)

In [ ]:
# some slicing examples

full_array = np.array([1, 1, 4, 5, 9, 10])

no_first = full_array[1:]     # cut off the first element of full_array
no_last = full_array[:-1]     # cut off the last element
every_other = full_array[::2] # view every other element
start_stop_step = full_array[1:5:2]  # start at index 1, stop at index 5 (exclusive), and step in intervals of 2
print(no_first, no_last, every_other, start_stop_step)

full_array[1:5:2] = full_array[1:5:2] * 2   # can use slices to modify array elements in-place
print(full_array)

### Strategy 3: don't be afraid to use higher-dimensional arrays

In [ ]:
# example: generate a Binomial distribution via Monte-Carlo simulation

p = 0.5  # probability of success
n = 100  # number of trials per experiment/simulation
jmax = 1000  # number of simulations to run

In [ ]:
%%timeit

# Kipping's code

# Define lists we will need using comprehension
trial = [i+1 for i in range(n)]  # define an integer list
frac = [0 for i in range(n)]     # define an empty list

for j in range(jmax): # big loop for each simulation
    k = 0   # counter for the number of successes achieved
    
    for i in range(n):
        uniform = np.random.uniform()
        if uniform < p:         # if uniform < p, then we have a success
            k = k+1             # increase counter to reflect success
        frac[i] = float(k)/float(trial[i])
        
    # append results to an array using vertical stack
    if j > 0:
        results_kipping = np.vstack([results_kipping,frac])
    else:
        results_kipping = np.array(frac)

In [ ]:
%%timeit

# my code

rand_probs = np.random.uniform(size=(jmax, n))
trial_nums = np.arange(1, n+1)
results_me = np.cumsum(rand_probs > p, axis=1) / trial_nums

In [ ]:
# if you feed pyplot a 2D array, it'll plot each of the columns separately; .T (transpose) flips rows and columns

plt.plot(np.tile(trial_nums, (jmax,1)).T, results_me.T, color="black", alpha=0.1)
plt.xlabel("number of flips")
plt.ylabel("fraction of heads")
plt.ylim(0,1)
plt.xlim(0,100)

# fraction of heads approaches normal distribution as number of flips grows
plt.show()

## Vectorizing user-defined functions

In [ ]:
# example 1: turn our previous routine for finding factors into a function

def count_factors(n):
    
    if (n <= 0): return 0
    
    remainders = n % possible_factors  
    factors_np = possible_factors[np.where(remainders == 0)]
    
    return factors_np.size


# vectorize our function to make it compatible with array inputs

count_factors_vec = np.vectorize(count_factors)  

In [ ]:
ns = np.arange(1000)    # generate a list of numbers

# use the vectorized function to apply count_factors to every element in ns
t1 = time.time()
count_factors_vec(ns)   
t2 = time.time()

# use a loop to apply count_factors to every element in ns
count_factors_list = []
t3 = time.time()
for n in ns:
    count_factors_list.append(count_factors(n))  
t4 = time.time()

print(f"Vectorized time: {t2-t1}; Loop time: {t4-t3}")  # not a very big time difference; vectorize is mostly for convenience

# Further reading

NumPy API reference: https://numpy.org/doc/stable/reference/index.html

Using multidimensional array: http://www.math.buffalo.edu/~badzioch/MTH337/PT/PT-multidimensional_numpy_arrays/PT-multidimensional_numpy_arrays.html

Broadcasting: https://www.tutorialspoint.com/numpy/numpy_broadcasting.htm, 
https://numpy.org/doc/stable/user/basics.broadcasting.html

NumPy array manipulation routines: https://numpy.org/doc/stable/reference/routines.array-manipulation.html

Linear algebra in NumPy: https://numpy.org/doc/stable/reference/routines.linalg.html

# 3. Useful Scipy functions (curve_fit, fsolve, interpolate)
## scipy.optimize.curve_fit

In [ ]:
from scipy.optimize import curve_fit 


In [ ]:
#define Planck's function
def BB_lambdaT(l, T):
  
    #constants (SI units)
    hplanck =6.62607015e-34 #m^2kg/s
    clight =299792458. #m/s
    kb =1.380649e-23 #m^2kg/(s^2K^1)

    return (2.*hplanck*clight**2/l**5)/(np.exp(hplanck*clight/(l*kb*T))-1.)


In [ ]:
#make some data (blackbody + noise)

factor=1.e12
lambda_sun=np.linspace(100.e-9, 2000.e-9, 100) #wavelength range [m]
T_sun=5778 # Sun's temp [K]
BB_sun=BB_lambdaT(lambda_sun, T_sun)
noise=factor*np.random.normal(size=len(lambda_sun)) #sample noise from a gaussian distribution
BB_sun_data=BB_sun+noise

#what happens to our fit when we increase noise (i.e. factor variable)?

In [ ]:
#unpack curve_fit outputs
BB_params, BB_cov = curve_fit(f=BB_lambdaT, xdata=lambda_sun, ydata=BB_sun_data, p0=1000.)

#if you have known errors on your data, you could also input them into curve_fit 
# via sigma and absolute_sigma parameters
# other useful things to input are bounds on your parameters

#fitted parameters (i.e. temperature)
print(BB_params)

#estimated covariance matrix
print(BB_cov)

#diagonal terms of the covariance matrix are our parameter variances
#therefore, one-sigma errors on parameters are square root of the diagonal terms of the covariance matrix

#1 sigma error in our temperature
BB_params_stdev=np.sqrt(np.diag(BB_cov))
print(BB_params_stdev)



In [ ]:
#make an array of the fitted values
BB_sun_fit=BB_lambdaT(lambda_sun, BB_params[0])


In [ ]:
#just rounding up the parameters for the plot legend
temp=int(np.round(BB_params[0],0))
temp_sigma=int(np.round(BB_params_stdev[0],0))

In [ ]:
#now let's plot both our data & the fit
plt.figure()
plt.scatter(lambda_sun*1.e9, BB_sun_data/1.e12, marker='o',label='data', color='magenta')
plt.plot(lambda_sun*1.e9, BB_sun_fit/1.e12, label=f'curve_fit: T$_\odot$={temp}$\pm${temp_sigma}', color='cyan')
plt.legend()
plt.xlabel('wavelength [nm]')
plt.ylabel(r'intensity [kW/m$^{2}$/nm]')

### additional notes:

questions for discussion: When can curve_fit be useful? What are its limitations?

resources: 

curve_fit uses non-linear least squares: 
https://en.wikipedia.org/wiki/Non-linear_least_squares

some resources to learn/review about the covariance matrix:
https://mathworld.wolfram.com/Covariance.html
https://towardsdatascience.com/understanding-the-covariance-matrix-92076554ea44


## scipy.interpolate.interpolate1d

In [ ]:
from scipy.interpolate import interp1d

In [ ]:
#define some sparse data
#let's use power law since it's common to encounter in astronomy :)
x_values=np.array([0,4,11,40,75,100], dtype=np.float64)
powerlaw_curve=x_values**0.3

#now let's create interpolator functions using different methods 
nearest = interp1d(x_values, powerlaw_curve, kind='nearest') #nearest neighbor
linear = interp1d(x_values, powerlaw_curve, kind='linear') #linear spline
cubic = interp1d(x_values, powerlaw_curve, kind='cubic') #cubic spline

In [ ]:
#now lets take some well-sampled x-array
x_values_interp=np.linspace(1,100,100)

In [ ]:
#and interpolate for those values
powerlaw_curve_nearest=nearest(x_values_interp)
powerlaw_curve_linear=linear(x_values_interp)
powerlaw_curve_cubic=cubic(x_values_interp)

In [ ]:
#plot our sparse data
plt.figure()
plt.scatter(x_values, powerlaw_curve, marker='d',label='data',s=50)
plt.legend()
plt.xlabel('x')
plt.ylabel(r'x$^{0.3}$')

In [ ]:
#plot our interpolated data for three methods
plt.figure()
plt.scatter(x_values, powerlaw_curve, marker='d',label='data')
plt.scatter(x_values_interp, powerlaw_curve_nearest, marker='x',label='nearest')
plt.scatter(x_values_interp, powerlaw_curve_linear, marker='*',label='linear')
plt.scatter(x_values_interp, powerlaw_curve_cubic, marker='o',label='cubic')
plt.legend()
plt.xlabel('x')
plt.ylabel(r'x$^{0.3}$')

### additional notes:
questions for discussion: how do you choose your interpolator? when are interpolators useful?

other helpful functions: scipy.interpolate.interp2d, scipy.interpolate.griddata


## scipy.optimize.fsolve

In [ ]:
from scipy.optimize import fsolve

#define a function that you want to find the roots of
def func(x):
    
    return (x-10.11)*(x+1.15)*(x-4.6)*x


In [ ]:
#plot it
xs=np.linspace(-20,20,100)
plt.plot(xs, func(xs))
plt.hlines(0,-20,20, ls='--')
# plt.ylim([-500,1000])
# plt.xlim([-5,15])

In [ ]:
#solve for roots with initial guesses
roots=fsolve(func, x0=[-2,0,3,9])
roots

In [ ]:
#solve for roots with bad initial guesses
roots=fsolve(func, x0=[-20,30,5,1000])
roots

In [ ]:
#solve for roots but decrease running time
roots=fsolve(func, x0=[-2,0,3,9],maxfev=10)
roots


### additional notes:
questions for discussion: when can fsolve be useful?
other useful functions: root 

# Writing and reading files with Astropy and NumPy
## Astropy tables

In [ ]:
from astropy.table import Table

In [ ]:
#define some particle data: their IDs and 3-D position coordinates

pID=np.array([1, 2, 3, 4, 5])
x=np.array([2., 4., 6., 8., 10.])
y=np.array([1., 4., 9., 16., 25.])
z=np.array([1., 1., 1., 1., 1.])

particle_info='particle positions \n particle ID, x, y, z' #define file comment string


In [ ]:
#now let's create and write a table to a file 
particle_table=Table([pID, x, y, z], names=['pID','x', 'y','z'])
particle_table.write('particles.csv', format='ascii.csv', overwrite=True)  


In [ ]:
%%time
particle_read=Table.read('particles.csv', format='ascii.csv')

In [ ]:
#for large files, setting guess=False when reading a file can noticeably decrease computational time

## numpy.savetxt and numpy.loadtxt

In [ ]:
#save arrays and comment at the top of the file
np.savetxt('particles.txt', (pID, x, y, z), fmt='%1.1f', header=particle_info)
particles_read_txt=np.loadtxt('particles.txt', dtype=np.float64)
print(particles_read_txt)
print(particles_read_txt[0,:])

#arrays are saved as rows

In [ ]:
#let's stack our arrays as columns instead
np.savetxt('particles.txt', np.column_stack([pID, x, y, z]), fmt="%1.1f", header=particle_info)
particles_read_txt=np.loadtxt('particles.txt', dtype=np.float64)
print(particles_read_txt)
print(particles_read_txt[:,0])
#now arrays are saved as columns

In [ ]:
#we can also unpack the txt file into separate arrays
IDs, x_pos, y_pos, z_pos=np.loadtxt('particles.txt', dtype=np.float64, unpack=True)
print(IDs)

In [ ]:
#lets add more particles
add_ID=np.arange(6,10,1)
add_x=add_y=add_z=np.arange(1,5,1)
print(add_ID)
print(add_x)
np.savetxt('particles.txt', np.column_stack([add_ID, add_x, add_y, add_z]), fmt="%1.1f")

In [ ]:
#lets check the file
IDs, x_pos, y_pos, z_pos=np.loadtxt('particles.txt', dtype=np.float64, unpack=True)
print(IDs)

In [ ]:
#lets append to the file instead 
with open('particles.txt','a') as particle_file:
    np.savetxt(particle_file, np.column_stack([add_ID, add_x, add_y, add_z]), fmt="%1.1f")


In [ ]:
#now let's check again
IDs, x_pos, y_pos, z_pos=np.loadtxt('particles.txt', dtype=np.float64, unpack=True)
print(IDs)

In [ ]:
#what happens if we explicitly open the file
particle_file=open('particles.txt','w')
np.savetxt(particle_file, np.column_stack([add_ID, add_x, add_y, add_z]), fmt="%1.1f")
#particle_file.close()

### additional notes:
see also numpy.genfromtxt -- useful when some data is missing
see also functions in the pandas package

# 4. Check your work with units and constants in AstroPy

What is inside astropy's `units` submodule?  Quoting the [documentation](https://docs.astropy.org/en/stable/units/index.html),

> `astropy.units` handles defining, converting between, and performing arithmetic with physical quantities, such as meters, seconds, Hz, etc. It also handles logarithmic units such as magnitude and decibel.
> 
> `astropy.units` does not know spherical geometry or sexagesimal (hours, min, sec): if you want to deal with celestial coordinates, see the `astropy.coordinates` package.

We'll explore how to use astropy `units`, and how it can help us guard against bugs and mistakes.

## 4.1 Overview of astropy.units

Just a glimpse, not comprehensive -- there's more in the documentation and various online tutorials.

### Start by playing around with some built-in units, to see what's possible.

In [ ]:
from astropy import units as u

In [ ]:
u.m

In [ ]:
u.meter

In [ ]:
u.picometer

In [ ]:
u.picometer.to(u.m)  # ask audience: give me a unit!

In [ ]:
u.picometer.to(u.g)  # ask audience: give me another unit! (make sure to do both length & "not-length")

In [ ]:
force = 5 * u.m * u.kg / u.s**2

In [ ]:
force

In [ ]:
type(force)

In [ ]:
type(force.value)

In [ ]:
force.value

In [ ]:
force.unit

In [ ]:
force.cgs

In [ ]:
force.si

## 4.2 Doing math with units

Let's return to the Planck blackbody function as a motivating example.

$$
    B_{\lambda}(\lambda, T)
    = \frac{2 h c^2}{\lambda^5}
    \frac{1}{\exp\left(\frac{hc}{\lambda k_B T} -1\right)}
$$

with Planck's constant $h$, speed of light $c$, and Boltzmann constant $k_B$.

Planck's function, expressed in per-wavelength form ($B_\lambda$ rather than $B_\nu$), has units of radiance per wavelength.  That is power per steradian per area per wavelength.  Phew!

### How would we program this function with units?  How does having units help us?

Note: we need to attach the "per-steradian" piece manually.  Angular units like radians and steradians are dimensionless, and steradians don't already show up in the constants $h$, $c$, $k_B$, nor in our inputs $\lambda$, $T$.

In [ ]:
def BB_lambdaT(l, T):
    #constants (SI units)
    hplanck=6.62607015e-34 #m^2kg/s
    clight=299792458. #m/s
    kb=1.380649e-23 #m^2kg/(s^2K^1)
    return (2.*hplanck*clight**2/l**5)/(np.exp(hplanck*clight/(l*kb*T))-1.)

In [ ]:
from astropy import constants as const

In [ ]:
const.h

In [ ]:
const.c

In [ ]:
const.k_B

In [ ]:
def BB_lambdaT(l, T):
    hplanck = const.h
    clight = const.c
    kb = const.k_B
    return (2.*hplanck*clight**2/l**5)/(np.exp(hplanck*clight/(l*kb*T))-1.) / u.sr

### Now, if we forget to put in units, we get a `UnitConversionError`.

In [ ]:
BB_lambdaT(400e-9, 5778)

### With units, it seems to work!

In [ ]:
BB_lambdaT(400*u.nm, 5778*u.K)

### Let's try converting it to the expected form, power per solid angle per area per wavelength.

In [ ]:
BB_lambdaT(400*u.nm, 5778*u.K).to(u.Watt/u.sr/u.m**2/u.m)

### What if we accidentally swap the order of our inputs?

Units will give an extra hint that something is amiss.  You may not have memorized the typical magnitude of the Sun's spectral radiance, but you might more easily recognize that $\mathrm{Kelvin}^5$ looks fishy in the denominator.

In [ ]:
BB_lambdaT(5778*u.K, 400*u.nm)

### The units submodule works seamlessly with numpy and matplotlib.

In [ ]:
lambda_sun = np.linspace(100e-9, 2000e-9, 100) * u.m #wavelength range [m]
T_sun = 5778 * u.K # sun's temp [K]
sun_BB = BB_lambdaT(lambda_sun, T_sun)

In [ ]:
plt.plot(lambda_sun, sun_BB)
plt.show()

### The units submodule has a nice auto-labeling feature with matplotlib, too.

In [ ]:
from astropy import visualization

In [ ]:
astropy.visualization.quantity_support()

In [ ]:
plt.plot(lambda_sun, sun_BB)
plt.show()

In [ ]:
plt.plot(lambda_sun.to(u.nm), sun_BB)
plt.axvspan(4000*u.Angstrom, 7000*u.Angstrom, facecolor='g', alpha=0.2)
plt.show()

### Now that we have a nice, clean working function, let's add a "docstring" to finish the job.

In [ ]:
def BB_lambdaT(l, T):
    """
    Planck's blackbody function per unit wavelength
    Reference: Zombeck, Handbook of Space Astronomy and Astrophysics, 2nd Ed., page 268.
    
    Arguments:
        l: wavelength in astropy units (length), scalar or array
        T: temperature in astropy units (Kelvin), scalar or array
    Returns:
        value(s) of Planck's function evaluated at (l, T) with astropy units attached
    """
    hplanck = const.h
    clight = const.c
    kb = const.k_B
    return (2.*hplanck*clight**2/l**5)/(np.exp(hplanck*clight/(l*kb*T))-1.) / u.sr

## 4.3 Extra: a little more "defensive programming"

We've seen that astropy units can help us write correct, concise, and more readable code, and we've seen that the units submodule works cleanly with `numpy` and `matplotlib`.

__Exercise:__ what if we give correct units, but unphysical values to the blackbody function?  Try modifying the function `BB_lambdaT(...)` to prevent the user from inputting negative values.  You might use: `assert`, `raise Exception`, `np.any(...)`, `np.all(...)`.

As you modify the function, make sure it works on previous input (both scalar and array).  This is called "regression testing", to check that new changes to your function don't cause it to regress in other unexpected ways.

In [ ]:
BB_lambdaT(-100*u.m, -1000*u.K)

### Let's add assertion checks for `l` and `T`

In [ ]:
def BB_lambdaT(l, T):
    """
    Planck's blackbody function per unit wavelength
    Reference: Zombeck, Handbook of Space Astronomy and Astrophysics, 2nd Ed., page 268.
    
    Arguments:
        l: wavelength in astropy units (length), scalar or array. Must be >= 0.
        T: temperature in astropy units (Kelvin), scalar or array. Must be >= 0.
    Returns:
        value(s) of Planck's function evaluated at (l, T) with astropy units attached
    """
    assert l >= 0
    assert T >= 0
    hplanck = const.h
    clight = const.c
    kb = const.k_B
    return (2.*hplanck*clight**2/l**5)/(np.exp(hplanck*clight/(l*kb*T))-1.) / u.sr

### The function throws an error for negative scalar inputs, as desired.

In [ ]:
BB_lambdaT(-100*u.m, -1000*u.K)

### Verify that the function works on previous input by trying to re-generate the plot we made above.

In [ ]:
lambda_sun = np.linspace(100e-9, 2000e-9, 100) * u.m #wavelength range [m]
T_sun = 5778 * u.K # sun's temp [K]
sun_BB = BB_lambdaT(lambda_sun, T_sun)

### Woops, we did break something!  Let's fix it.

The code `l >= 0` returns a numpy array of Booleans.  As the error message says, the truth value of an array with more than one message is ambiguous: should we treat `[True, False, False, True, True]` as `True` or `False`??  Let's guard against _any_ negative input values for wavelength or temperature by using `np.any(...)`, similar to what is suggested.

Instead of `np.any(l >= 0)`, notice that we could also write `(l >= 0).any()`.

In [ ]:
def BB_lambdaT(l, T):
    """
    Planck's blackbody function per unit wavelength
    Reference: Zombeck, Handbook of Space Astronomy and Astrophysics, 2nd Ed., page 268.
    
    Arguments:
        l: wavelength in astropy units (length), scalar or array. Must be >= 0.
        T: temperature in astropy units (Kelvin), scalar or array. Must be >= 0.
    Returns:
        value(s) of Planck's function evaluated at (l, T) with astropy units attached
    """
    assert np.any(l >= 0)
    assert np.any(T >= 0)
    hplanck = const.h
    clight = const.c
    kb = const.k_B
    return (2.*hplanck*clight**2/l**5)/(np.exp(hplanck*clight/(l*kb*T))-1.) / u.sr

In [ ]:
lambda_sun = np.linspace(100e-9, 2000e-9, 100) * u.m #wavelength range [m]
T_sun = 5778 * u.K # sun's temp [K]
sun_BB = BB_lambdaT(lambda_sun, T_sun)

In [ ]:
plt.plot(lambda_sun.to(u.nm), sun_BB)
plt.show()

### Our function passes the "regression test".  Let's re-confirm that it gives the correct error for either negative scalar or array input.

In [ ]:
lambda_sun = np.linspace(100e-9, 2000e-9, 100) * u.m #wavelength range [m]
T_sun = -1 * 5778 * u.K # sun's temp [K]
sun_BB = BB_lambdaT(lambda_sun, T_sun)

In [ ]:
BB_lambdaT(-100*u.m, -1000*u.K)

## 4.4 Some general remarks

Defensive programming is not a cure-all!  Bugs can come in other ways.  It's also possible to be too careful and to clutter your code needlessly.  Some personal taste / practice is involved.

Why should we use astropy units specifically?  What other tools can help us perform calculations with units?
* Wolfram|Alpha, for quick one-off calculations with units: https://www.wolframalpha.com/
* `yt` and `unyt` provide unit-aware interfaces for simulation outputs: https://yt-project.org/
* Many other specialized packages will provide their own unit packages.  Different packages / systems may not talk easily with one another (but, can be aided by projects like `yt`).  Example from a well-used simulation code: https://arepo-code.org/wp-content/userguide/parameterfile.html#system-of-units


## 4.5 References

* Documentation: https://docs.astropy.org/en/stable/units/index.html
* Tutorials on astropy units specifically:
  + (recommended) https://learn.astropy.org/tutorials/quantities.html
  + https://astropy4cambridge.readthedocs.io/en/latest/_static/Astropy%20-%20Unit%20Conversion.html

# 5. Parting thoughts on scientific computing, generally


* Programming in Python, and working with command-line software in general, are skills learned by lots of trial and error, Googling for answers, and __asking friends and mentors__.  More senior mentors may not be as up-to-date on current software trends and development: peers, graduate students, postdocs can be good resources.  Such skills can also be learned in CS classes, which can be especially helpful if you are interested in more computational aspects of astro.  They will go farther than what is strictly needed for most day-to-day astro work.


* Many community organizations run one- or multi-day workshops for scientific computing skills, and may have some curricula online.
  + [Software Carpentry](https://software-carpentry.org/)
  + [Flatiron Sciware](http://sciware.flatironinstitute.org/)
  + [Code/Astro](https://semaphorep.github.io/codeastro/), recently run at the AAS Summer Meeting


* You often don't have to re-invent the wheel!  Many, many libraries and built-in functions exist.


* Visualization is a deep subject on its own.  One starting point: think about accessibility for colorblind viewers.


* Mathematica notebooks are another powerful tool, especially for symbolic math.  Downsides: proprietary/closed source, cost.  You can get a free license at https://www.cuit.columbia.edu/content/mathematica.  Mathematica appeared in 1988, before Python's first release in 1991(!), and helped pave the way towards IPython notebooks / Jupyter.


* Coding style: https://google.github.io/styleguide/pyguide.html. Much is not applicable for scientists, and much is a matter of taste (unlike big teams working on shared code, where people need to agree on a consistent style).  But you may find some useful nuggets.  For example,
  + good practices for docstrings and comments
  + a suggestion to keep functions <= 40 lines (a suggestion that I routinely break myself!..)
  + 2.19 "Avoid power features"
  + 3.16 "Naming"


* Some suggested best practices in scientific computing: https://journals.plos.org/plosbiology/article?id=10.1371/journal.pbio.1001745